In [ ]:
import time
import threading

import numpy as np
import scipy as sp
from matplotlib import pyplot as plt
from scipy import linalg

import pandas as pd
import pandas_datareader.data as web

In [ ]:
from sklearn import linear_model
import pandas as pd

In [ ]:
d=2
M=150

def nilpotent(M):
    B = np.zeros((M,M))
    for i in range(2,M):
        B[i,i-1]=1.0
    return B

def canonical(i,M):
    e = np.zeros((M,1))
    e[i,0]=1.0
    return e

def vectorfieldSABR(state,increment):
    return np.array([(np.sqrt(state[0]**2))**0.7*np.exp(-0.5*state[1]),0.1])*increment[0]+np.array([0,0.25*(state[0]+state[1])])*increment[1]

def vectorfield2d(state,increment):
    return np.array([(2.0*np.sqrt(state[1]**2))**0.7+np.sin(state[1]),1.0*state[1]+np.cos(state[1])])*increment[0]+np.array([(2.0*np.sqrt(state[1]**2))**0.7,0.0*state[1]])*increment[1]

def randomAbeta(d,M):
    A = []
    beta = []
    for i in range(d):
        B = 0.0*nilpotent(M) + np.random.normal(0.0,0.03,size=(M,M))
        B = np.random.permutation(B)
        A = A + [B]
        beta = beta + [0.0*canonical(i,M)+np.random.normal(0.0,0.03,size=(M,1))]
    return [A,beta]

In [ ]:
Abeta = randomAbeta(d,M)
A = Abeta[0]
beta = Abeta[1]

def sigmoid(x):
    return np.tanh(x)

def reservoirfield(state,increment):
    value = np.zeros((M,1))
    for i in range(d):
        value = value + sigmoid(np.matmul(A[i],state) + beta[i])*increment[i]
    return value

In [ ]:
class SDE:
    def __init__(self,timehorizon,initialvalue,dimension,dimensionBM,dimensionR,vectorfield,timesteps):
        self.timehorizon = timehorizon
        self.initialvalue = initialvalue # np array
        self.dimension = dimension
        self.dimensionBM = dimensionBM
        self.dimensionR = dimensionR
        self.vectorfield = vectorfield
        self.timesteps = timesteps

    def path(self):
        BMpath = [np.zeros(self.dimensionBM)]
        SDEpath = [np.array([1.0, self.initialvalue])]
        for i in range(self.timesteps):
            helper = np.random.normal(0,np.sqrt(self.timehorizon/self.timesteps),self.dimensionBM)
            BMpath = BMpath + [BMpath[-1]+helper]
            SDEpath = SDEpath + [np.exp(-1.0*self.timehorizon/self.timesteps)*(SDEpath[-1]+self.vectorfield(SDEpath[-1],helper))]

        return [BMpath, SDEpath]

    def anypath(self):
        BMpath = [np.zeros(self.dimensionBM)]
        SDEpath = [np.array([1.0, self.initialvalue])]#[np.ones((self.dimension,1))*self.initialvalue]

        for i in range(self.timesteps):
            helper = np.cos(BMpath[-1]*50)*self.timehorizon/self.timesteps#np.random.normal(0,np.sqrt(self.timehorizon/self.timesteps),self.dimensionBM)
            BMpath = BMpath + [BMpath[-1]+helper]
            SDEpath = SDEpath + [np.exp(-0.0*self.timehorizon/self.timesteps)*(SDEpath[-1]+self.vectorfield(SDEpath[-1],helper))]

        return [BMpath, SDEpath]

    def reservoir(self,BMpath,scaling,k,moving_time):
        reservoirpath = [canonical(k,self.dimensionR)*self.initialvalue]
        for i in range(moving_time-1):
            increment = scaling*(BMpath[i+1]-BMpath[i])
            reservoirpath = reservoirpath + [np.exp(-1.0*self.timehorizon/self.timesteps)*(reservoirpath[-1]+reservoirfield(reservoirpath[-1],increment))]
        return reservoirpath

In [ ]:
Abeta = randomAbeta(d,M)
A = Abeta[0]
beta = Abeta[1]

#### Generate stock prices

In [ ]:
# Number of seconds/hours in one trading day: 9.30-16.30
sec = 150 #25200
min = 420

In [ ]:
SDE_of_interest = SDE(7,1.0,2,d,M,vectorfield2d,sec)
#SDE_of_interest = SDE(7,1.0,2,d,M,vectorfieldSABR,10000)
training = SDE_of_interest.path()

In [ ]:
f1,p1=plt.subplots(2,1,figsize=(6,6),sharey=True)
p1[0].plot(training[0],'r')
p1[1].plot(training[1],'g')
#plt.savefig('trainingpath.pdf')
plt.show()

In [ ]:
available_SDE_data_list = []
available_BM_data_list = []

available_BM_data = training[0][0]
available_SDE_data = training[1][0]

In [ ]:
print(type(training[0][0]))

In [ ]:
print(len(training[0]))

In [ ]:
train = False

In [ ]:
def read_data_line_by_line(data_SDE, data_BM):
    global available_BM_data
    global available_SDE_data
    global train
    
    for i in range(1, len(training[0])): # len of list
        print(np.shape(available_BM_data))
        print(np.shape(available_SDE_data))

        available_BM_data_list.append(data_BM[i])
        available_SDE_data_list.append(data_SDE[i])

        available_BM_data = np.array(available_BM_data_list)
        available_SDE_data = np.array(available_SDE_data_list)

        if i % 10 == 0:
            train = True
        else: 
            train = False

        time.sleep(1) 


def train_model():
    global available_BM_data
    global available_SDE_data
    global train

    while train:
        if (len(available_SDE_data) % 10) == 0: 

            X = SDE_of_interest.reservoir(available_BM_data,1,0, len(available_SDE_data)) 
            Xtrain = np.squeeze(X) 

            Ytrain = np.squeeze(available_SDE_data) 

            lm = linear_model.Ridge(alpha=0.05)
            model = lm.fit(Xtrain, Ytrain)
            plt.plot(model.predict(Xtrain),'r')
            plt.plot(Ytrain,'b')
            plt.show()
            model.score(Xtrain,Ytrain)
            model.coef_

            print("Model trained successfully")

In [ ]:
# Start the data reading thread
data_thread = threading.Thread(target=read_data_line_by_line, args=(training[1], training[0]))
data_thread.start()

# Start the model training thread
train_thread = threading.Thread(target=train_model)
train_thread.start()

# Join threads to ensure they complete (in a real scenario, you may not need to join if you want continuous execution)
data_thread.join()
train_thread.join()


In [ ]:
train_lock = threading.Lock()

def read_data(data_BM, data_SDE):
    global available_BM_data
    global available_SDE_data

    for i in range(1, len(data_BM)):

        available_BM_data_list.append(data_BM[i])
        available_SDE_data_list.append(data_SDE[i])

        available_BM_data = np.array(available_BM_data_list)
        available_SDE_data = np.array(available_SDE_data_list)
        time.sleep(1)  # Simulating the 1-second interval
        
        # Check if we need to train/retrain the model
        if len(available_SDE_data_list) % 20 == 0:
            threading.Thread(target=train_model).start()

def train_model():
    global model
    with train_lock:
        # Wait until there are at least 20 elements
        #while len(available_elements) < 20:
        #    time.sleep(0.1)
        
        # Train the model with the available data
        X = SDE_of_interest.reservoir(available_BM_data,1,0, len(available_SDE_data)) 
        Xtrain = np.squeeze(X) 

        Ytrain = np.squeeze(available_SDE_data) 

        lm = linear_model.Ridge(alpha=0.05)
        model = lm.fit(Xtrain, Ytrain)
        
        len_predict = len(available_BM_data) + 20 # predict 20 timesteps into the future
        X_pred = training[0][:len_predict]
        X_pred = np.squeeze(X_pred)
        plt.plot(model.predict(X_pred),'r')
        plt.plot(Ytrain,'b')
        plt.show()
        #model.score(Xtrain,Ytrain)
        #model.coef_

In [ ]:
# Start the data reading thread
data_thread = threading.Thread(target=read_data, args=(training[0], training[1]))
data_thread.start()
data_thread.join()

print("All data read. Final available elements:")
print(available_SDE_data)

I have the following problem. For the moment I have a brownian motion on a fixed time interval [a, b]. The I train my model on some interval [a, c] with c < a. After this I can predict the stock price on [c, b] but for this I need the brownian motion (and then the reservoir) on [c, b] first. But this I don't have in real time analysis, since I can only calculate the underlying brownian motion once I have the SDE path. Solutions:\
\
1) I can use a random brownian motion and then see how my model learns. This does not work very well...\
\
2) Using the same method, I could learn the relation f(SDE[:t]) -> SDE[t:t+alpha]\
\
3) Or starting with an uncomplete (exponential) brownian motion can I predict it in the future? If yes can I learn this relation